In [3]:
import cv2
import numpy as np
import sys
sys.path.append("../functions")
from irls import irls_fit

video_path = "../data/solidWhiteRight.mp4"
cap = cv2.VideoCapture(video_path)

# Output setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('../data/output_IRLS.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # --- Preprocess ---
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # --- Get Edge Points ---
    y_coords, x_coords = np.where(edges > 0)

    # --- Right lane ---
    mask_r = (y_coords > frame.shape[0] * 0.6) & (x_coords > frame.shape[1] * 0.6)
    x_r = x_coords[mask_r]
    y_r = y_coords[mask_r]
    if len(x_r) > 50:
        idx_r = np.random.choice(len(x_r), min(300, len(x_r)), replace=False)
        beta_r = irls_fit(y_r[idx_r], x_r[idx_r])
        y_fit = np.linspace(frame.shape[0]*0.6, frame.shape[0], 100)
        Xy = np.vstack([y_fit**i for i in range(2, -1, -1)]).T
        x_fit = Xy @ beta_r
        pts = np.array([np.stack([x_fit, y_fit], axis=1)], dtype=np.int32)
        cv2.polylines(frame, pts, False, (0, 0, 255), 3)

    # --- Left lane ---
    mask_l = (
    (y_coords > frame.shape[0] * 0.6) &
    (y_coords < frame.shape[0] * 0.95) &
    (x_coords > frame.shape[1] * 0.1) &
    (x_coords < frame.shape[1] * 0.45)
    )

    x_l = x_coords[mask_l]
    y_l = y_coords[mask_l]

    if len(x_l) > 50 and np.std(x_l) > 20:
        idx_l = np.random.choice(len(x_l), min(300, len(x_l)), replace=False)
        beta_l = irls_fit(y_l[idx_l], x_l[idx_l])
        x_fit = Xy @ beta_l
        pts = np.array([np.stack([x_fit, y_fit], axis=1)], dtype=np.int32)
        cv2.polylines(frame, pts, False, (255, 0, 0), 3)

    
    # --- Save and show ---
    out.write(frame)
    cv2.imshow('Lane Detection (IRLS)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
